In [ ]:
# If we are on colab: this clones the repo and installs the dependencies
from pathlib import Path

if Path.cwd().name != "LightGlue":
    !git clone --quiet https://github.com/cvg/LightGlue/
    %cd LightGlue
    !pip install --progress-bar off --quiet -e .

from lightglue import LightGlue, SuperPoint, DISK
from lightglue.utils import load_image, rbd
from lightglue import viz2d

import torch

torch.set_grad_enabled(False)
from pathlib import Path

images = Path("lung-lobes_1/scale-5pc")

In [ ]:
from lightglue import LightGlue, SuperPoint, DISK
from lightglue.utils import load_image, rbd
from lightglue import viz2d
import torch
import pandas as pd
import os
base_dir = '/home/20223176/LightGlue/'
image_paths = []

# 디렉토리 순회
for root, dirs, files in os.walk(base_dir):
    for subdir in dirs:
        scale_dir = os.path.join(root, subdir)
        png_files = [f for f in os.listdir(scale_dir) if f.endswith(".png")]

        # 이미지 경로 추가
        for png_file in png_files:
            image_paths.append(os.path.join(scale_dir, png_file))

# 기존 image_paths 기반으로 추가 경로 생성
new_image_paths = []
scales = ["scale-50pc", "scale-25pc", "scale-10pc", "scale-5pc"]

for path in image_paths:
    if "scale-100pc" in path:
        # 다른 스케일 경로 생성
        for scale in scales:
            new_path = path.replace("scale-50pc", scale)
            new_image_paths.append(new_path)

# 모든 경로 합치기
all_image_paths = image_paths + new_image_paths

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 'mps', 'cpu'

extractor = SuperPoint(max_num_keypoints=1024).eval().to(device)  # load the extractor
matcher = LightGlue(features="superpoint").eval().to(device)

In [ ]:
image0 = load_image(images / "29-039-U-35W-Izd1-1-HE.jpg")
image1 = load_image(images / "29-039-U-35W-Izd1-6-ki67.jpg")

feats0 = extractor.extract(image0.to(device))
feats1 = extractor.extract(image1.to(device))
matches01 = matcher({"image0": feats0, "image1": feats1})
feats0, feats1, matches01 = [
    rbd(x) for x in [feats0, feats1, matches01]
]  # remove batch dimension

kpts0, kpts1, matches = feats0["keypoints"], feats1["keypoints"], matches01["matches"]
m_kpts0, m_kpts1 = kpts0[matches[..., 0]], kpts1[matches[..., 1]]


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 이미지 로드

image = cv2.imread(images / "29-039-U-35W-Izd1-1-HE.jpg", cv2.IMREAD_GRAYSCALE)
stack = [1, 3, 5] 
plt.figure(figsize=(15, 10))

for i in range(len(stack)):
    blurred_image = cv2.GaussianBlur(image, (31, 31), stack[i])

    sobel_x = cv2.Sobel(blurred_image, cv2.CV_64F, 1, 0, ksize=3) 
    sobel_y = cv2.Sobel(blurred_image, cv2.CV_64F, 0, 1, ksize=3) 
    sobel_x = cv2.convertScaleAbs(sobel_x)
    sobel_y = cv2.convertScaleAbs(sobel_y)
    
    sobel_magnitude = cv2.magnitude(sobel_x.astype(np.float32), sobel_y.astype(np.float32))
    sobel_magnitude = cv2.normalize(sobel_magnitude, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    log = cv2.Laplacian(blurred_image, cv2.CV_64F)  
    log = cv2.convertScaleAbs(log)  

    combined_response = sobel_magnitude + log

    keypoints = m_kpts0.cpu().numpy() 
    keypoints_int = keypoints.astype(int)
    
    height, width = combined_response.shape
    valid_keypoints = [(x, y) for x, y in keypoints_int if 0 <= x < width and 0 <= y < height]

    response_values = [combined_response[y, x] for x, y in valid_keypoints]

    sorted_keypoints = sorted(zip(valid_keypoints, response_values), key=lambda x: x[1], reverse=True)

    N = 50
    top_keypoints = sorted_keypoints[:N]

    image_color = cv2.imread(images / "29-039-U-35W-Izd1-1-HE.jpg")

    plt.subplot(1, 3, i + 1)
    plt.imshow(cv2.cvtColor(image_color, cv2.COLOR_BGR2RGB))
    plt.title(f"Keypoints with Gaussian Blur_Sobel_LoG_sigma_{stack[i]}")
    for (x, y), response_value in top_keypoints:
        plt.scatter(x, y, c="red", s=10)  # Keypoints 표시
    keypoints_image_path = f"output/result/sobel/keypoints_with_blur_sobel_log_sigma_{stack[i]}.jpg"
    plt.axis("off")
    plt.savefig(keypoints_image_path, bbox_inches="tight", pad_inches=0)

plt.show()

